# Data Preparation and Exploratory Data Analysis

In [2]:
# Mounting Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Installing Dependencies

!pip install xformers
!pip install transformers[torch]
!pip install datasets
!pip install  torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [4]:
# Loading Required Libraries

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
import time
import copy

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import math
import random
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import gc
import torchtext
from tqdm import tqdm

# Misc.
import warnings
warnings.filterwarnings('ignore')



In [5]:
# Hyperparameter Values for Training

batch_size = 16
MAX_LEN = 160
RANDOM_SEED = 16
MODEL_NAME = 'bert-base-cased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 20


In [6]:
# Checking device type

device

device(type='cuda')

# Bert

In [7]:
# Loading the dataset from drive into a Pandas dataframe

df = pd.read_csv("/content/drive/MyDrive/NLP4RE-data/Expanded_PROMISE.csv")

In [8]:
# Checking first 10 rows of the dataframe

df.head(10)

,Id,RequirementText,Class,Binary_Label
0,1,The product shall be available during normal b...,A,1
1,2,The product shall be available for use 24 hour...,A,1
2,2,Out of 1000 accesses to the system the system ...,A,1
3,3,The system shall be available for use between ...,A,1
4,3,The system shall achieve 95% up time.,A,1
5,5,The product shall adhere to the corporate onli...,A,1
6,5,The product shall achieve a 98% uptime. The pr...,A,1
7,6,Aside from server failure the software produc...,A,1
8,8,The website shall be available for use 24 hour...,A,1
9,8,The website shall achieve 99.5% up time.,A,1


In [9]:
# Checking for null values

df.isnull().sum()

Id                 0
RequirementText    0
Class              0
Binary_Label       0
dtype: int64

In [10]:
# Dataframe metadata information

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Id               969 non-null    int64 
 1   RequirementText  969 non-null    object
 2   Class            969 non-null    object
 3   Binary_Label     969 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 30.4+ KB


In [11]:
# Distribution of the dataframe

df['Binary_Label'].value_counts()

1    525
0    444
Name: Binary_Label, dtype: int64

In [12]:
# Build a BERT based tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
# Determining maximum sequence length in the dataset

tokens = []

max_ = float("-inf")

# Iterate through the content slide
for txt in df["RequirementText"]:
    t = np.array(tokenizer.encode(txt))

    max_ = max(max_ , len(t))

    tokens.append(t)

tokens = np.array(tokens)

print(max_)

110


In [14]:
print(tokens.shape)

(969,)


In [15]:
# Creating a Torch dataset class

class GPReviewDataset(Dataset):
    # Constructor Function
    def __init__(self, reviews, targets, tokenizer, max_len = 160):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    # Length magic method
    def __len__(self):
        return len(self.reviews)

    # get item magic method
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        # Encoded format to be returned
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [16]:
# Creating a train and validation set from data

df_train, df_val= train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)


In [17]:
# Distribution of train and validation set

df_train['Binary_Label'].value_counts() , df_val['Binary_Label'].value_counts()

(1    414
 0    361
 Name: Binary_Label, dtype: int64,
 1    111
 0     83
 Name: Binary_Label, dtype: int64)

In [18]:
# Function to initialize loader for training

def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = GPReviewDataset(
        reviews=df["RequirementText"].to_numpy(),
        targets=df["Binary_Label"].to_numpy(),
        tokenizer=tokenizer,

    )

    return DataLoader(
        ds,
        batch_size=batch_size,

    )

In [19]:
# Data Loaders

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, batch_size)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, batch_size)


In [20]:
data = next(iter(train_data_loader))

print(data)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'review_text': ['The system shall allow user to choose their preferable topping.', 'The product shall display each type of ship in a grid using an image of the specific type of ship.', 'The Disputes applications shall interface with the Merchant Information Database. The Merchant Information Database provides detailed information with regard to the merchant. All merchant detail information shall be obtained from the Merchant Information Database.', 'The system shall display only 10 matching result on the current screen.', 'Can give feedback to the 24X7 Customer Care Service center about their impression for the site and services.', 'The users should be able to easily use the system to successfully complete their budgets  within the stipulated time for completion. With a week of training prior to product implementation  98% of the users shall complete their budgets within the stipulated time.', 'For each turn  the product shall allow the offensive player to define a shot.', 'The websit

In [21]:
data['input_ids']

tensor([[  101,  1109,  1449,  ...,     0,     0,     0],
        [  101,  1109,  3317,  ...,     0,     0,     0],
        [  101,  1109, 12120,  ...,     0,     0,     0],
        ...,
        [  101,  1109, 14274,  ...,     0,     0,     0],
        [  101,  1109,  1440,  ...,     0,     0,     0],
        [  101,  1109,  2593,  ...,     0,     0,     0]])

In [22]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

torch.Size([16, 160])
torch.Size([16, 160])
torch.Size([16])


In [23]:
# Build the Binary Classifier class
class BinaryClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes = 2):
        super(BinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)


    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        #  Add a dropout layer
        output = self.drop(pooled_output)
        return self.out(output)

In [24]:
sentiment_int = {'functional' : 0, 'non-functional': 1}

In [25]:
model = BinaryClassifier(len(sentiment_int))
model = model.to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [26]:
model

BinaryClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [33]:
# Function for a single training epoch
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples,bert = True):



    model = model.train()
    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)


        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        # Backward prop
        loss.backward()

        # Gradient Descent
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)


def eval_model(model, data_loader, loss_fn, device, n_examples, f1=0):

    model = model.eval()
    losses = []
    correct_predictions = 0
    total_targets = []
    total_preds = []
    with torch.no_grad():
        for d in tqdm(data_loader):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            # Get model ouptuts
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
            total_targets.extend(targets.cpu())
            total_preds.extend(preds.cpu())

        if f1==1:
          print(classification_report(total_targets, total_preds))

    return correct_predictions.double() / n_examples, np.mean(losses)

In [28]:
# Train function

def train(model,EPOCHS = 10):

  print()
  print()
  print()
  print(model)
  print()
  print()
  print()


  optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

  total_steps = len(train_data_loader) * EPOCHS

  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=0,
      num_training_steps=total_steps
  )

  # Set the loss function
  loss_fn = nn.CrossEntropyLoss().to(device)

  # history = defaultdict(list)
  best_accuracy = 0

  for epoch in range(EPOCHS):


      print(f'Epoch {epoch + 1}/{EPOCHS}')
      print('-' * 20)

      train_acc, train_loss = train_epoch(
      model,
      train_data_loader,
      loss_fn,
      optimizer,
      device,
      scheduler,
      len(df_train)
    )



      val_acc, val_loss = eval_model(
      model,
      val_data_loader,
      loss_fn,
      device,
      len(df_val)
    )


      print(f' train_loss {train_loss:.5f}  train_acc {train_acc:.5f} <-> val_loss {val_loss:.5f} val_accuracy {val_acc:.5f}')
      print()

# Bert Training

In [29]:
train(model,EPOCHS = 20)




BinaryClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

100%|██████████| 13/13 [00:01<00:00,  8.43it/s]


 train_loss 0.56360  train_acc 0.70710 <-> val_loss 0.36451 val_accuracy 0.85052

Epoch 2/20
--------------------


100%|██████████| 13/13 [00:01<00:00,  8.29it/s]


 train_loss 0.26304  train_acc 0.91742 <-> val_loss 0.43002 val_accuracy 0.89691

Epoch 3/20
--------------------


...

 train_loss 0.00010  train_acc 1.00000 <-> val_loss 0.89916 val_accuracy 0.91237

Epoch 19/20
--------------------


100%|██████████| 13/13 [00:01<00:00,  7.63it/s]


 train_loss 0.00009  train_acc 1.00000 <-> val_loss 0.90083 val_accuracy 0.91237

Epoch 20/20
--------------------


100%|██████████| 13/13 [00:01<00:00,  7.70it/s]

 train_loss 0.00009  train_acc 1.00000 <-> val_loss 0.90143 val_accuracy 0.91237



In [34]:
# Validation set results

loss_fn = nn.CrossEntropyLoss().to(device)
val_acc, val_loss = eval_model(
      model,
      val_data_loader,
      loss_fn,
      device,
      len(df_val),
      f1=1
    )
print(f' Val_loss {val_loss:.5f} Val_Accuracy {val_acc:.5f}')

100%|██████████| 13/13 [00:01<00:00,  6.98it/s]

              precision    recall  f1-score   support

           0       0.88      0.92      0.90        83
           1       0.94      0.91      0.92       111

    accuracy                           0.91       194
   macro avg       0.91      0.91      0.91       194
weighted avg       0.91      0.91      0.91       194

 Val_loss 0.90143 Val_Accuracy 0.91237


# Transformer

In [35]:
df.head()

,Id,RequirementText,Class,Binary_Label
0,1,The product shall be available during normal b...,A,1
1,2,The product shall be available for use 24 hour...,A,1
2,2,Out of 1000 accesses to the system the system ...,A,1
3,3,The system shall be available for use between ...,A,1
4,3,The system shall achieve 95% up time.,A,1


In [36]:
df_train, df_val

(     Id                                    RequirementText Class  Binary_Label
 337  33  The system shall allow user to choose their pr...     F             0
 528  10  The product shall display each type of ship in...    LF             1
 591   4  The Disputes applications shall interface with...     O             1
 427  46  The system shall display only 10 matching resu...     F             0
 351  38  Can give feedback to the 24X7 Customer Care Se...     F             0
 ..   ..                                                ...   ...           ...
 321  30  Enable the admin to generate reports which con...     F             0
 581   3  The system shall able to operate within a busi...     O             1
 121   4  The Disputes System must provide a confirmatio...     F             0
 238   9  The leads washing functionality will compile b...     F             0
 681   8  System shall let existing customers log into t...    PE             1
 
 [775 rows x 4 columns],
      Id     

In [37]:
df2 = df.copy()

In [38]:
def map_label(label):
    if label == 0:
        return 'function'
    elif label == 1:
        return 'non-function'
    else:
        return 'unknown'

# Apply the mapping function to create a new column
df2['category'] = df2["Binary_Label"].apply(map_label)

In [39]:
df2.head()

,Id,RequirementText,Class,Binary_Label,category
0,1,The product shall be available during normal b...,A,1,non-function
1,2,The product shall be available for use 24 hour...,A,1,non-function
2,2,Out of 1000 accesses to the system the system ...,A,1,non-function
3,3,The system shall be available for use between ...,A,1,non-function
4,3,The system shall achieve 95% up time.,A,1,non-function


In [40]:
df_train2, df_val2= train_test_split(df2, test_size=0.2, random_state=RANDOM_SEED)

In [41]:
trg_langs = df_train2["category"].unique().tolist()
trg_langs


['function', 'non-function']

In [42]:
df_train

,Id,RequirementText,Class,Binary_Label
337,33,The system shall allow user to choose their pr...,F,0
528,10,The product shall display each type of ship in...,LF,1
591,4,The Disputes applications shall interface with...,O,1
427,46,The system shall display only 10 matching resu...,F,0
351,38,Can give feedback to the 24X7 Customer Care Se...,F,0
...,...,...,...,...
321,30,Enable the admin to generate reports which con...,F,0
581,3,The system shall able to operate within a busi...,O,1
121,4,The Disputes System must provide a confirmatio...,F,0
238,9,The leads washing functionality will compile b...,F,0


In [43]:
class LangDataset(Dataset):
    def __init__(self, ds, trg_langs, train_vocab=None):
        self.corpus = ds

        if not train_vocab:
            self.src_vocab, self.trg_vocab = self._build_vocab()
        else:
            self.src_vocab, self.trg_vocab = train_vocab

    def __len__(self):
        return len(self.corpus)

    def __getitem__(self, item):
        text = self.corpus.iloc[item].RequirementText
        lang = self.corpus.iloc[item].category

        return {
            'src': self.src_vocab.lookup_indices(text.lower().split()),
            'trg': self.trg_vocab.lookup_indices([lang])
        }

    def _build_vocab(self):
        # Here one could remove stopwords and use word lemmatisation.
        # Both techniques will reduce the vocab size and hence model size
        # and could also enhance the model's performance
        src_tokens = self.corpus.RequirementText.str.cat().lower().split()

        src_vocab = build_vocab_from_iterator([src_tokens], specials=["<unk>", "<pad>"])
        src_vocab.set_default_index(src_vocab['<unk>'])

        trg_vocab = build_vocab_from_iterator([trg_langs])

        return src_vocab, trg_vocab

In [44]:
def collate_fn(batch, pad_value, device):
    trgs = []
    srcs = []
    for row in batch:
        srcs.append(torch.tensor(row["src"], dtype=torch.long).to(device))
        trgs.append(torch.tensor(row["trg"]).to(device))

    padded_srcs = pad_sequence(srcs, padding_value=pad_value)
    return {"src": padded_srcs, "trg": torch.tensor([trgs]).to(device)}

train_langds = LangDataset(df_train2, trg_langs)
test_langds = LangDataset(df_val2, trg_langs, (train_langds.src_vocab, train_langds.trg_vocab))

SRC_PAD_IDX = train_langds.src_vocab["<pad>"]

train_dt = DataLoader(train_langds, batch_size= batch_size, shuffle=
                   True, collate_fn=lambda batch_size: collate_fn(batch_size, SRC_PAD_IDX, device))

test_dt = DataLoader(test_langds, batch_size=batch_size, shuffle=
                   True, collate_fn=lambda batch_size: collate_fn(batch_size, SRC_PAD_IDX, device))


hyp_params = {
    "batch_size": batch_size,
    "lr": 0.0005,
    "num_epochs": EPOCHS,
    "d_model": 512, # Input embedding dimension
    "n_head": 8, # No. of multi-head attention block (aka paralle self-attention layers)
    "n_layers": 3,
    "feedforward_dim": 128,
    "dropout": 0.1
}


hyp_params["src_vocab_size"] = len(train_langds.src_vocab)
hyp_params["trg_vocab_size"] = len(trg_langs)

In [45]:
next(iter(train_dt))

{'src': tensor([[   2,   46,    2,    2,    2,    2,    0,    2,  103,    2,    2,    2,
             2,   83,   58,    2],
         [  10,    3,   10,    7,   10,   47,   14,    7,   14,    7,   37,    7,
           757,  461,  886,    7],
         [   3,   21,    3,    3,    3,   26,    6,    3,    6,    3,   47,    3,
           966,    8,  332,    3],
         [   6,    2,    6,  107,   29,    6,   16,  217,   16,   29,    3,   29,
             5,  121,    3,   38],
         [  16,  433,   16,    8,    2,  965,    4,    2,    4,    2,   54,    2,
             2,  137,   45,   41],
         [   4,   97,    4,   12,  565,   18,   42,   12,  816, 1662, 1163, 2045,
           156,   28,    6, 2038],
         [   6,    4,  106,   93,   32, 1846,   18,  400,    2,  178,  407,   19,
             7,  359,   16,    4],
         [ 428,  739,  130,    4,    8,    9,    2, 1916,  225,  367,   23,   44,
            14,  184,    4, 2581],
         [  13,    8,  278,  427, 1411,    0,  560,  914,

In [46]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, maxlen = 5000):
        super(PositionalEncoding, self).__init__()

        # A tensor consists of all the possible positions (index) e.g 0, 1, 2, ... max length of input
        # Shape (pos) --> [max len, 1]
        pos = torch.arange(0, maxlen).unsqueeze(1)
        pos_encoding = torch.zeros((maxlen, d_model))

        sin_den = 10000 ** (torch.arange(0, d_model, 2)/d_model) # sin for even item of position's dimension
        cos_den = 10000 ** (torch.arange(1, d_model, 2)/d_model) # cos for odd

        pos_encoding[:, 0::2] = torch.sin(pos / sin_den)
        pos_encoding[:, 1::2] = torch.cos(pos / cos_den)

        # Shape (pos_embedding) --> [max len, d_model]
        # Adding one more dimension in-between
        pos_encoding = pos_encoding.unsqueeze(-2)
        # Shape (pos_embedding) --> [max len, 1, d_model]

        self.dropout = nn.Dropout(dropout)

        # We want pos_encoding be saved and restored in the `state_dict`, but not trained by the optimizer
        # hence registering it!
        # Source & credits: https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723/2
        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, token_embedding):
        # shape (token_embedding) --> [sentence len, batch size, d_model]

        # Concatenating embeddings with positional encodings
        # Note: As we made positional encoding with the size max length of sentence in our dataset
        #       hence here we are picking till the sentence length in a batch
        #       Another thing to notice is in the Transformer's paper they used FIXED positional encoding,
        #       there are methods where we can also learn them
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])


class InputEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(InputEmbedding, self).__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, tokens):
        # shape (tokens) --> [sentence len, batch size]
        # shape (inp_emb) --> [sentence len, batch size, d_model]
        # Multiplying with square root of d_model as they mentioned in the Transformer's paper
        inp_emb = self.embedding(tokens.long()) * math.sqrt(self.d_model)
        return inp_emb


class TransformerClassifier(nn.Module):
    def __init__(self,
                  src_vocab_size,
                 trg_vocab_size ,
                 d_model,
                 dropout,
                 n_head,
                 dim_feedforward,
                 n_layers,
                ):
        super().__init__()

        self.src_inp_emb = InputEmbedding(src_vocab_size, d_model)
        self.trg_inp_emb = InputEmbedding(trg_vocab_size, d_model)

        self.positional_encoding = PositionalEncoding(d_model, dropout=dropout)

        # Only using Encoder of Transformer model
        encoder_layers = nn.TransformerEncoderLayer(d_model, n_head, dim_feedforward, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)

        self.d_model = d_model
        self.decoder = nn.Linear(d_model, trg_vocab_size)

    def forward(self, x):
        x_emb = self.positional_encoding(self.src_inp_emb(x))
        # Shape (output) -> (Sequence length, batch size, d_model)
        output = self.transformer_encoder(x_emb)
        # We want our output to be in the shape of (batch size, d_model) so that
        # we can use it with CrossEntropyLoss hence averaging using first (Sequence length) dimension
        # Shape (mean) -> (batch size, d_model)
        # Shape (decoder) -> (batch size, d_model)
        return self.decoder(output.mean(0))

In [47]:
def train_model(model, train_dataloader, criterion, optimizer , n_examples ):
    model.train()
    epoch_loss = 0
    correct_predictions = 0
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        # Clear the accumulating gradients
        optimizer.zero_grad()

        src = batch["src"]  # shape --> [seq len, batch size]
        trg = batch["trg"]  # shape --> [1, batch size]

        # shape (out) --> [batch size, trg size]
        out = model(src)
        _, preds = torch.max(out, dim=1)
        correct_predictions += torch.sum(preds == trg)
        loss = criterion(out, trg.squeeze(0))

        loss.backward()

        optimizer.step()
        epoch_loss += loss.detach().cpu()

    return correct_predictions.double() / n_examples, epoch_loss/len(train_dataloader)


def evaluate_model(model, valid_dataloader, criterion , n_examples):
    model.eval()
    epoch_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_dataloader):
            src = batch["src"]  # shape --> [seq len, batch size]
            trg = batch["trg"]  # shape --> [1, batch size]

            # shape (out) --> [batch size, trg size]
            out = model(src)
            _, preds = torch.max(out, dim=1)
            correct_predictions += torch.sum(preds == trg)
            loss = criterion(out, trg.squeeze(0))

            epoch_loss += loss.detach().cpu()



    return correct_predictions.double() / n_examples, epoch_loss/len(valid_dataloader)




In [48]:
model2 = TransformerClassifier(  hyp_params["src_vocab_size"],
                                hyp_params["trg_vocab_size"] ,
                                hyp_params["d_model"],
                                hyp_params["dropout"],
                                hyp_params["n_head"],
                                hyp_params["feedforward_dim"],
                                hyp_params["n_layers"]
                                ).to(device)


criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=hyp_params["lr"])

print(model2)

TransformerClassifier(
  (src_inp_emb): InputEmbedding(
    (embedding): Embedding(2602, 512)
  )
  (trg_inp_emb): InputEmbedding(
    (embedding): Embedding(2, 512)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear

In [49]:
hyp_params

{'batch_size': 16,
 'lr': 0.0005,
 'num_epochs': 20,
 'd_model': 512,
 'n_head': 8,
 'n_layers': 3,
 'feedforward_dim': 128,
 'dropout': 0.1,
 'src_vocab_size': 2602,
 'trg_vocab_size': 2}

In [50]:


def train2(model,EPOCHS = EPOCHS):


  for epoch in range(EPOCHS):
    start = time.time()
    gc.collect()
    torch.cuda.empty_cache()

    train_acc,train_loss = train_model(model, train_dt, criterion, optimizer,len(df_train2))
    val_acc,val_loss = evaluate_model(model, test_dt, criterion,len(df_val2))

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    print(f' train_loss {train_loss:.5f}  train_acc {train_acc:.5f} <-> Val_loss {val_loss:.5f} val_accuracy {val_acc:.5f}')



# Transformer Training

In [51]:
train2(model2, 20)


# {'batch_size': 16,
#  'lr': 0.0005,
#  'num_epochs': 20,
#  'd_model': 768,
#  'n_head': 12,
#  'n_layers': 12,
#  'feedforward_dim': 128,
#  'dropout': 0.1,
#  'src_vocab_size': 2602,
#  'trg_vocab_size': 2}

100%|██████████| 49/49 [00:00<00:00, 69.28it/s]


Epoch 1/20
----------
 train_loss 0.71563  train_acc 0.49032 <-> Val_loss 0.73256 val_accuracy 0.43814


100%|██████████| 49/49 [00:00<00:00, 77.09it/s]


Epoch 2/20
----------
 train_loss 0.71658  train_acc 0.48129 <-> Val_loss 0.74653 val_accuracy 0.42268


100%|██████████| 49/49 [00:00<00:00, 77.83it/s]


...

Epoch 18/20
----------
 train_loss 0.71888  train_acc 0.47742 <-> Val_loss 0.73811 val_accuracy 0.42784


100%|██████████| 49/49 [00:00<00:00, 63.62it/s]


Epoch 19/20
----------
 train_loss 0.71213  train_acc 0.48645 <-> Val_loss 0.73577 val_accuracy 0.43814


100%|██████████| 49/49 [00:00<00:00, 57.27it/s]


Epoch 20/20
----------
 train_loss 0.71625  train_acc 0.48516 <-> Val_loss 0.73908 val_accuracy 0.43814


In [ ]:
train2(model2, 40)


# {'batch_size': 16,
#  'lr': 0.0005,
#  'num_epochs': 20,
#  'd_model': 512,
#  'n_head': 8,
#  'n_layers': 3,
#  'feedforward_dim': 128,
#  'dropout': 0.1,
#  'src_vocab_size': 2602,
#  'trg_vocab_size': 2}

100%|██████████| 49/49 [00:00<00:00, 69.06it/s]


Epoch 1/40
----------
 train_loss 0.71055  train_acc 0.48129 <-> Val_loss 0.68619 val_accuracy 0.53608


100%|██████████| 49/49 [00:00<00:00, 74.51it/s]


Epoch 2/40
----------
 train_loss 0.70949  train_acc 0.46065 <-> Val_loss 0.69335 val_accuracy 0.54124


100%|██████████| 49/49 [00:00<00:00, 72.51it/s]


...

Epoch 38/40
----------
 train_loss 0.71250  train_acc 0.47355 <-> Val_loss 0.69570 val_accuracy 0.52062


100%|██████████| 49/49 [00:00<00:00, 73.39it/s]


Epoch 39/40
----------
 train_loss 0.71549  train_acc 0.45935 <-> Val_loss 0.70002 val_accuracy 0.51031


100%|██████████| 49/49 [00:00<00:00, 70.19it/s]


Epoch 40/40
----------
 train_loss 0.71353  train_acc 0.47355 <-> Val_loss 0.69067 val_accuracy 0.53093
